In [ ]:
# Install required libraries if necessary
!pip install pandas_datareader yfinance

# Import libraries
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_datareader.data as web
import statsmodels.api as sm
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
# Define the date range
start_date = '2022-01-01'
end_date = '2025-01-01'

# Fetch Fama-French factors
ff_factors = web.DataReader('F-F_Research_Data_Factors_daily', 'famafrench', start_date, end_date)[0]

# Display the first few rows
print("Fama-French Daily Factors:")
print(ff_factors.head())

In [ ]:
# Rename columns for easier reference
ff_factors = ff_factors.rename(columns={'Mkt-RF': 'MKT', 'SMB': 'SMB', 'HML': 'HML', 'RF': 'RF'})

In [ ]:
# Define the stock ticker
stock_ticker = 'ASB'

In [ ]:
# Download stock data
stock_data = yf.download(stock_ticker, start=start_date, end=end_date)

# Determine which column to use for returns calculation
if 'Adj Close' in stock_data.columns:
    price_column = 'Adj Close'
elif 'Close' in stock_data.columns:
    price_column = 'Close'
    print("'Adj Close' not available. Using 'Close' prices instead.")
else:
    raise KeyError("Neither 'Adj Close' nor 'Close' found in the stock data.")

# Calculate stock returns
stock_data['Return'] = stock_data[price_column].pct_change()
stock_returns = stock_data['Return'].dropna()

In [ ]:
# Prepare data
stock_returns = stock_returns.reset_index()
stock_returns.rename(columns={'Date': 'Date', 'Return': 'Stock_Return'}, inplace=True)
ff_factors = ff_factors.reset_index()

# Convert Date columns to datetime
stock_returns['Date'] = pd.to_datetime(stock_returns['Date'])
ff_factors['Date'] = pd.to_datetime(ff_factors['Date'])

# Merge datasets
data = pd.merge(stock_returns, ff_factors, on='Date')

In [ ]:
# Calculate excess returns
data['Excess_Return'] = data['Stock_Return'] - data['RF']/100

In [ ]:
# Define dependent and independent variables
Y = data['Excess_Return']
X = data[['MKT', 'SMB', 'HML']]

# Add a constant
X = sm.add_constant(X)

In [ ]:
# Fit the Fama-French Three-Factor Model
ff_model = sm.OLS(Y, X).fit()
print(ff_model.summary())

In [ ]:
# Extract coefficients and R-squared
alpha_ff = ff_model.params['const']
beta_mkt = ff_model.params['MKT']
beta_smb = ff_model.params['SMB']
beta_hml = ff_model.params['HML']
r_squared_ff = ff_model.rsquared

# Display the results
print("\nFama-French Three-Factor Model Results:")
print(f"Alpha (Intercept): {alpha_ff}")
print(f"Beta (MKT): {beta_mkt}")
print(f"Beta (SMB): {beta_smb}")
print(f"Beta (HML): {beta_hml}")
print(f"R-squared: {r_squared_ff}")

R- squared:                                                                  CAPM - 27.9% where as Fama-French is 69%                                 
SMB Coefficient (Size Effect)

The SMB coefficient is 0.0094. This positive coefficient suggests that ASB's stock returns are positively related to the returns of small-cap stocks relative to large-cap stocks.
Interpretation: A positive SMB beta indicates that ASB tends to behave more like a small-cap stock and is expected to outperform when small-cap stocks (as a group) outperform large-cap stocks. This is consistent with the "size effect" or "small firm effect," where smaller companies are often observed to generate higher returns than larger companies over the long term.
HML Coefficient (Value Effect)

The HML coefficient is 0.0134. This positive coefficient indicates a positive relationship between ASB's stock returns and the returns of value stocks (high book-to-market ratio) relative to growth stocks (low book-to-market ratio).
Interpretation: A positive HML beta implies that ASB exhibits characteristics of a value stock. It tends to perform well when value stocks outperform growth stocks, aligning with the "value effect" where value stocks are often found to have higher returns compared to growth stocks over time.
In summary, for ASB stock:

Size Effect: The positive SMB coefficient suggests a positive size effect, indicating that the stock's returns are positively influenced by the performance of small-cap stocks.
Value Effect: The positive HML coefficient suggests a positive value effect, meaning the stock's returns are positively influenced by the performance of value stocks.                                            
 Copilot was used to assist with code